In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
def infonce_loss(z1: torch.Tensor, z2: torch.Tensor, temperature: float = 0.07) -> torch.Tensor:
    """
    InfoNCE 손실 함수 (SimCLR 방식).
    Args:
        z1: 첫 번째 뷰의 프로젝션 (B, D)
        z2: 두 번째 뷰의 프로젝션 (B, D)
        temperature: 소프트맥스 온도 조절 파라미터
    Returns:
        InfoNCE 손실 값 (스칼라)
    """
    # L2 정규화
    z1 = F.normalize(z1, dim=-1)
    z2 = F.normalize(z2, dim=-1)

    # 두 뷰를 합침 -> (2B, D)
    z = torch.cat([z1, z2], dim=0)
    batch_size = z1.size(0)

    # 유사도 행렬 계산 (2B, 2B)
    similarity_matrix = z @ z.t()

    # 안정화를 위해 각 행의 최대값 빼주기 (선택 사항이지만 권장)
    similarity_matrix = similarity_matrix - similarity_matrix.max(dim=1, keepdim=True).values

    # 자기 자신과의 유사도(대각선) 제외 마스크
    mask = torch.eye(2 * batch_size, device=z.device, dtype=torch.bool)

    # 포지티브 쌍의 인덱스 생성 (z1[i]의 짝은 z2[i], z2[i]의 짝은 z1[i])
    labels = torch.cat([torch.arange(batch_size) + batch_size, torch.arange(batch_size)], dim=0).to(z.device)

    # 자기 자신과의 유사도를 매우 작은 값으로 채움
    similarity_matrix = similarity_matrix.masked_fill(mask, -float('inf'))

    # CrossEntropyLoss 계산 (logit = sim / temp)
    loss = F.cross_entropy(similarity_matrix / temperature, labels)
    return loss

In [ ]:
def barlow_twins_loss(z1: torch.Tensor, z2: torch.Tensor, lambda_param: float = 5e-3, eps: float = 1e-8) -> torch.Tensor:
    """
    Barlow Twins 손실 함수.
    Args:
        z1: 첫 번째 뷰의 프로젝션 (B, D)
        z2: 두 번째 뷰의 프로젝션 (B, D)
        lambda_param: off-diagonal 요소에 대한 가중치
        eps: 표준편차 계산 시 분모가 0이 되는 것을 방지하기 위한 작은 값
    Returns:
        Barlow Twins 손실 값 (스칼라)
    """
    batch_size, dimension = z1.shape

    # 1. 배치 방향으로 정규화 (각 특징 채널의 평균=0, 표준편차=1)
    z1_norm = (z1 - z1.mean(0)) / (z1.std(0) + eps)
    z2_norm = (z2 - z2.mean(0)) / (z2.std(0) + eps)

    # 2. 교차 상관 행렬 계산 (D, D)
    cross_corr_matrix = (z1_norm.T @ z2_norm) / batch_size

    # 3. 손실 계산
    # 3.1. 대각선 요소(on-diagonal): 1이 되도록 유도
    on_diag = torch.diagonal(cross_corr_matrix).add_(-1).pow_(2).sum()
    # 3.2. 비-대각선 요소(off-diagonal): 0이 되도록 유도
    off_diag = cross_corr_matrix.clone().fill_diagonal_(0).pow_(2).sum()

    # 최종 손실 = 대각선 손실 + 람다 * 비-대각선 손실
    loss = on_diag + lambda_param * off_diag
    return loss

In [ ]:
def triplet_loss(z1: torch.Tensor, z2: torch.Tensor, margin: float = 1.0) -> torch.Tensor:
    """
    Triplet 손실 함수. 배치 내 다른 샘플을 네거티브로 사용.
    Args:
        z1: 앵커 뷰의 프로젝션 (B, D)
        z2: 포지티브 뷰의 프로젝션 (B, D)
        margin: 포지티브 쌍과 네거티브 쌍 간의 최소 거리 마진
    Returns:
        Triplet 손실 값 (스칼라)
    """
    # L2 정규화 (선택 사항이지만 거리를 일관되게 만듦)
    z1 = F.normalize(z1, dim=-1)
    z2 = F.normalize(z2, dim=-1)

    # 간단한 네거티브 샘플링: 포지티브 배치를 한 칸씩 밀어서 사용
    # z1[i]에 대한 네거티브는 z2[i-1]이 됨
    negative = z2.roll(shifts=1, dims=0)

    # PyTorch의 TripletMarginLoss 사용
    # loss = max(0, dist(anchor, positive) - dist(anchor, negative) + margin)
    loss = F.triplet_margin_loss(
        anchor=z1,
        positive=z2,
        negative=negative,
        margin=margin,
        p=2 # L2 (유클리드) 거리 사용
    )
    return loss

In [ ]:
def cosine_similarity_loss(z1: torch.Tensor, z2: torch.Tensor) -> torch.Tensor:
    """
    Cosine Similarity 손실 함수 (SimSiam 스타일).
    목표: 두 뷰 간의 코사인 유사도를 최대화 (-1 ~ 1 -> 손실 1 ~ -1)
    Args:
        z1: 첫 번째 뷰의 프로젝션 (B, D)
        z2: 두 번째 뷰의 프로젝션 (B, D)
    Returns:
        Cosine Similarity 손실 값 (스칼라)
    """
    # L2 정규화
    z1 = F.normalize(z1, dim=-1)
    z2 = F.normalize(z2, dim=-1)

    # 코사인 유사도 계산 (각 샘플별) 후 평균내고 (-) 부호 붙임
    # 유사도가 1일 때 손실이 -1, 유사도가 -1일 때 손실이 1이 됨
    loss = - F.cosine_similarity(z1, z2).mean()
    return loss